In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import glob
from sklearn.feature_selection import mutual_info_classif as MIC
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
import time

In [2]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.ensemble import RandomForestClassifier

In [3]:
# number = int(get_ipython().user_ns['param'])
number = 4

In [4]:
model_acc=[]
model_precision=[]
model_recal=[]
model_f1=[]
model_nam=[]
model_confusion=[]
model_time=[]
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report,f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
df = pd.read_csv('../Dataset CSV/csv_result-data'+str(number)+' Sampled Scenarios.csv')

In [6]:
# Perform the operation to create the new feature
df['R1-VCM1'] = (df['R1-PM1:V'] - df['R1-PM7:V']) / (df['R1-PM4:I'] - df['R1-PM10:I'])
df['R3-sV1'] = np.sin(df['R3-PA1:VH'] - df['R3-PA7:VH'])
df['R3-sV3'] = np.sin(df['R3-PA3:VH'] - df['R3-PA9:VH'])
# df['R2-VCM2'] = (df['R2-PM2:V'] - df['R2-PM8:V']) / (df['R2-PM5:I'] - df['R2-PM11:I'])
df['R2-VCM1'] = (df['R2-PM1:V'] - df['R2-PM7:V']) / (df['R2-PM4:I'] - df['R2-PM10:I'])
df['R3-VCM1'] = (df['R3-PM1:V'] - df['R3-PM7:V']) / (df['R3-PM4:I'] - df['R3-PM10:I'])
df['R4-sI1'] = np.sin(df['R4-PA4:IH'] - df['R4-PA10:IH'])
df['R2-VCA4'] = df['R2-PA7:VH'] - df['R2-PA10:IH']
df['R1-sI1'] = np.sin(df['R1-PA4:IH'] - df['R1-PA10:IH'])
df['R4-VCM2'] = (df['R4-PM2:V'] - df['R4-PM8:V']) / (df['R4-PM5:I'] - df['R4-PM11:I'])
df['R2-VCA1'] = np.sin(df['R2-PA1:VH'] - df['R2-PA4:IH'] - df['R2-PA7:VH'] - df['R2-PA10:IH'])
df['R3-VCM2'] = (df['R3-PM2:V'] - df['R3-PM8:V']) / (df['R3-PM5:I'] - df['R3-PM11:I'])
df['R1-VCA4'] = df['R1-PA7:VH'] - df['R1-PA10:IH']
df['R1-VCM2'] = (df['R1-PM2:V'] - df['R1-PM8:V']) / (df['R1-PM5:I'] - df['R1-PM11:I'])
df['R2-VCM2'] = (df['R2-PM2:V'] - df['R2-PM8:V']) / (df['R2-PM5:I'] - df['R2-PM11:I'])
df['R4-VCA4'] = df['R4-PA7:VH'] - df['R4-PA10:IH']
df['R4-VCM1'] = (df['R4-PM1:V'] - df['R4-PM7:V']) / (df['R4-PM4:I'] - df['R4-PM10:I'])

In [7]:
# add the marker to the last of the list
cols = list(df.columns)
cols.remove('marker')
df = df[cols + ['marker']]

In [8]:
df.replace([np.inf, -np.inf], np.nan, inplace= True)
df.isnull().sum().sum()

3209

In [9]:
from sklearn.impute import KNNImputer

# Create an instance of the KNNImputer class
imputer = KNNImputer(n_neighbors=1)

# Fit the imputer on the data
imputed_data = imputer.fit_transform(df)

# Replace the missing values in the original dataframe
df = pd.DataFrame(imputed_data, columns=df.columns)

In [10]:
print(df.isin([np.inf, -np.inf]).sum().sum(), df.isnull().sum().sum())

0 0


In [11]:
df_z_scaled = df.copy()
for column in df_z_scaled.columns:
#     print(df_z_scaled[column])
    if (df_z_scaled[column].std() != 0) and (column != 'id') and (column != 'marker'):
        df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()     
# print(df_z_scaled)

In [12]:
X = df_z_scaled.iloc[:,1:-1]
y = df_z_scaled.iloc[:,-1:]

In [15]:
# Get the names of the top 31 features
top_n_feature_names = ['R1-PM5:I', 'R1-VCM1', 'R3-PM5:I', 'R1-PA1:VH', 'R4-PA2:VH', 'R3-sV3',
 'R1-PA3:VH', 'R2-VCM1', 'R4-VCM1', 'R3-PA5:IH', 'R1-PA2:VH', 'R3-sV1',
 'R2-PA5:IH', 'R2-PA3:VH', 'R2-PA6:IH', 'R2-PA1:VH', 'R3-VCM1', 'R4-PA5:IH',
 'R2-VCA4', 'R3-PA6:IH', 'R4-PM2:V', 'R2-PA4:IH', 'R3-PA4:IH', 'R1-PA5:IH',
 'R1-sI1', 'R2-PA2:VH', 'R1-PM2:V', 'R3-PA:ZH', 'R2-VCM2', 'R1-VCM2', 'R4-VCM2', 'R1-PM2:V']

# Filter the original dataframe to only include the top n features
X_top_n = X[top_n_feature_names].copy()

X_new = X_top_n.copy()

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X_new, y, test_size=0.1, random_state=1)

In [17]:
from sklearn.ensemble import AdaBoostClassifier

# base_model = DecisionTreeClassifier(max_depth=5)

start_time = time.time()

clf_4= RandomForestClassifier(n_estimators=100, random_state=43)
# create the AdaBoost classifier
clf_ada_dt = AdaBoostClassifier(base_estimator=clf_4)

# train the model using the training data
clf_ada_dt.fit(X_train, y_train)

# make predictions on the testing data
y_pred = clf_ada_dt.predict(X_test)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

# evaluate the model's accuracy
score = accuracy_score(y_test, y_pred)

model_precision.append(precision_score(y_test,y_pred,average='weighted'))
model_recal.append(recall_score(y_test,y_pred,average='weighted'))
model_f1.append(f1_score(y_test,y_pred,average='weighted'))
print(f'Accuracy: {round(score*100,2)}%')
model_acc.append(score)
model_time.append(elapsed_time)
model_nam.append("AdaBoost")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Elapsed time: 3.08 seconds
Accuracy: 95.2%


In [18]:
data = {'Accuracy': model_acc, 'Precision': model_precision, 'Recall': model_recal, 'f1_score': model_f1, "elapsed_time": model_time}
df1 = pd.DataFrame(data)
print(df1)
# df1.to_csv('final_int_'+str(number)+'.csv', index=False)

   Accuracy  Precision    Recall  f1_score  elapsed_time
0  0.952015   0.956807  0.952015  0.952237      3.084744
